In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/DASANA

/content/drive/MyDrive/DASANA


In [ ]:
import urllib
import urllib.parse
import io

In [ ]:
from keras import preprocessing
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras import models
from keras import layers
from keras.layers import Embedding, Dense, Dropout, LSTM, Bidirectional
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential

In [ ]:
import re
import numpy as np
from numpy import array
import csv
import random
import pandas as pd
import re
from urllib.parse import unquote

data = pd.read_csv('csic_database.csv')
X = data[['Method','User-Agent','Pragma','Cache-Control', 'Accept','Accept-encoding','language', 'host', 'cookie', 'content-type', 'connection','lenght', 'content','URL']]
y = data['classification']

X_string = []
cols = X.columns
for i in range(len(X)):
    temp = ''
    for col in cols:
        temp += " " + str(X.iloc[i][col])
    ans = ''
    for t in temp:
        ans += t

    ans =  unquote(ans)

    ans = ans.lower()

    special_chars = ['/','&','=','+', ';', ',']
    for char in special_chars:
      ans = ans.replace(char, ' ')


    # Xóa các khoảng trắng thừa
    ans = ' '.join(ans.split())

    X_string.append(ans)

In [ ]:
print(X_string[154])

get mozilla 5.0 (compatible konqueror 3.5 linux) khtml 3.5.8 (like gecko) no-cache no-cache text xml application xml application xhtml xml text html q 0.9 text plain q 0.8 image png * * q 0.5 x-gzip x-deflate gzip deflate en localhost:8080 jsessionid 3e0d4e5cc38ee9b82904c365bbcb4c88 nan close nan nan http: localhost:8080 tienda1 publico miembros.jsp http 1.1


In [ ]:
tokenizer = Tokenizer(filters = '', num_words = 3000)
tokenizer.fit_on_texts(X_string)


sequences = tokenizer.texts_to_sequences(X_string)
pad = pad_sequences(sequences,padding='post')

print(len(pad))
print(len(pad[1]))
print(pad[0])

61065
92
[38 10 11 12 13 14 15 16 17 18 19  6  6  2  1  4  1  4 29  1  2 30  3 31
  2 32  3 33 34 35  7  7  3 36 20 21 22 23  9  8 24  5 25  5  5 26  8 37
 96 27 28  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
def da_sana_algorithm(dataset, sequences, f):

    au_dataset = []
    L = [len(req) for req in dataset]
    L_min = min(L)
    L_max = max(L)
    mean = (f + 2*f) / 2
    stddev = (2*f - f) / 4

    for request, Si  in zip(dataset, sequences ):

        length = len(request)
        L_normalized = (length - L_min) / (L_max - L_min)


        Ci = np.random.binomial(1, L_normalized*0.05 ,len(Si))

        Ni_Gauss = np.random.normal(mean, stddev, len(Si))
        Ni = abs(Ni_Gauss).astype(int)


        au_dataset.append(np.array(Si)*(Ci^1) + Ni*Ci)


    return au_dataset

In [ ]:
au_seq = np.array([*pad, *da_sana_algorithm(X_string,pad,3000)])
lable = np.array([*y,*y]).astype(np.int32)

print(au_seq.shape)
print(lable.shape)
print(type(lable[0]))

print(au_seq[3])

(122130, 92)
(122130,)
<class 'numpy.int32'>
[38 10 11 12 13 14 15 16 17 18 19  6  6  2  1  4  1  4 29  1  2 30  3 31
  2 32  3 33 34 35  7  7  3 36 20 21 22 23  9  8 24  5 25  5  5 26  8 37
 39 89 61 46 68 67 93 40 61 27 28  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]


In [ ]:
labels={'Normal':0, 'Anomalous':1}

In [ ]:
# split the data into training, validation, and testing sets
train_X, test_X, train_y, test_y = train_test_split(au_seq, lable, test_size=0.2, random_state=42)
train_X, val_X, train_y, val_y = train_test_split(train_X, train_y, test_size=0.25, random_state=42)

# size of the vector space in which characters will be embedded
embedding_dim = 16
max_chars = 10000

def build_model():
    model = Sequential()
    model.add(Embedding(max_chars, embedding_dim, input_length=train_X.shape[1]))
    model.add(Bidirectional(LSTM(units=128, activation='tanh', dropout=0.5, recurrent_dropout=0.5, return_sequences=True)))
    model.add(Bidirectional(LSTM(units=128, activation='tanh', dropout=0.5, recurrent_dropout=0.5)))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

model = build_model()
print(model.summary())

model.fit(train_X, train_y, epochs=10, batch_size=256, validation_data=(val_X, val_y))

test_loss, test_acc = model.evaluate(test_X, test_y)
print("Test Accuracy:", test_acc)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 92, 16)            160000    
                                                                 
 bidirectional (Bidirectiona  (None, 92, 256)          148480    
 l)                                                              
                                                                 
 bidirectional_1 (Bidirectio  (None, 256)              394240    
 nal)                                                            
                                                                 
 dense (Dense)               (None, 1)                 257       
                                                                 
Total params: 702,977
Trainable params: 702,977
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/10
287/287 [=========================

In [ ]:
labels={'Normal':0, 'Anomalous':1}

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report

# Make predictions on the test set
predictions = model.predict(test_X)
predicted_labels = [1 if pred > 0.5 else 0 for pred in predictions]

print(classification_report(test_y, predicted_labels, target_names=labels))

ConfusionMatrixDisplay.from_predictions(test_y, predicted_labels, display_labels= labels, xticks_rotation = 'vertical')